In [127]:
### importing modules and simulator class
%matplotlib inline

import autograd as ag
import autograd.numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from numpy.random import choice

class Simulator:

    """ Base class for simulators with access to joint score and joint likelihood ratios. """

    def __init__(self):
        pass

    def get_discretization(self):
        raise NotImplementedError

    def theta_defaults(self, n_thetas=100, single_theta=False, random=True):
        raise NotImplementedError()

    def rvs(self, theta, n, random_state=None):
        raise NotImplementedError()

    def rvs_score(self, theta, theta_score, n, random_state=None):
        raise NotImplementedError()

    def rvs_ratio(self, theta, theta0, theta1, n, random_state=None):
        raise NotImplementedError()

    def rvs_ratio_score(self, theta, theta0, theta1, theta_score, n, random_state=None):
        raise NotImplementedError()
        


In [3]:
dice_weights = [1/6,1/6,1/6,1/6,1/6,1/6]

def cl_markov_matrix(max_roll=6, jump_at_end=True):
    # Create the basic transition matrix:
    mat = np.zeros((101, 101))
    for i in range(101):
        mat[i + 1:i + 1 + max_roll, i] = 1. / max_roll
        

    mat[range(101), range(101)] += 1 - mat.sum(0)

    # account for the presence of chutes and ladders
    # we'll do this via  another transition matrix
    cl_mat = np.zeros((101, 101))
    ind = [CHUTES_LADDERS.get(i, i) for i in range(101)]
    cl_mat[ind, range(101)] = 1
    if jump_at_end:
        return cl_mat @ mat
    else:
        return mat @ cl_mat

# mat = cl_markov_matrix()
# plt.matshow(mat)

# plt.grid(False)

In [133]:
weights = np.array([1/6,1/6,1/6,1/6,1/6,1/6]) #theta parameter

CHUTES_LADDERS = {1:38, 4:14, 9:31, 16:6, 21:42, 28:84, 36:44,
                  47:26, 49:11, 51:67, 56:53, 62:19, 64:60,
                  71:91, 80:100, 87:24, 93:73, 95:75, 98:78} #dictionary of chutes and ladders



class Chutes_Ladders(Simulator):
    n=1
    ### initialization                    
    def __init__(self):
        pass
        
    
    
    ### creates samples x given theta (weights); p(x|theta)
    def simulate(self, theta=weights, n=1 , random_state=None): 
        turns_list = [] #number of turns per game
        winner = [] #which player wins each game
        rolls_list = []
        log_list = []
        
        position1_list = [] #player 1 position list
        position2_list = [] #player 2 position list
        final_position = []
        
        counter = 0
        
        
        while counter < n:
            ### setting up game
            log_p_xz = np.array([1.0])
            roll_list = []
            position = []
            position1 = 0
            position2 = 0
            turns = 0
            counter += 1
            while position1 < 100 and position2 < 100:
                turns += 1
                random_number1 = np.random.rand()
                random_number2 = np.random.rand()
                ### player_1's turn
                if random_number1 <= theta[0]:
                    roll1 = 1
                    log_p_xz *= theta[0]
                elif random_number1 <= theta[0] + theta[1]:
                    roll1 = 2
                    log_p_xz *= theta[1]
                elif random_number1 <= theta[0] + theta[1] + theta[2]:
                    roll1 = 3
                    log_p_xz *= theta[2]
                elif random_number1 <= theta[0] + theta[1] + theta[2] + theta[3]:
                    roll1 = 4
                    log_p_xz *= theta[3]
                elif random_number1 <= theta[0] + theta[1] + theta[2] + theta[3]+ theta[4]:
                    roll1 = 5
                    log_p_xz *= theta[4]
                else:
                    roll1 = 6
                    log_p_xz *= theta[5]
                
                position1 += roll1
                position1 = CHUTES_LADDERS.get(position1, position1)
                
                ### player_2's turn
                if random_number2 <= theta[0]:
                    roll2 = 1
                    log_p_xz *= theta[0]
                elif random_number2 <= theta[0] + theta[1]:
                    roll2 = 2
                    log_p_xz *= theta[1]
                elif random_number2 <= theta[0] + theta[1] + theta[2]:
                    roll2 = 3
                    log_p_xz *= theta[2]
                elif random_number2 <= theta[0] + theta[1] + theta[2] + theta[3]:
                    roll2 = 4
                    log_p_xz *= theta[3]
                elif random_number2 <= theta[0] + theta[1] + theta[2] + theta[3]+ theta[4]:
                    roll2 = 5
                    log_p_xz *= theta[4]
                else:
                    roll2 = 6
                    log_p_xz *= theta[5]
                
                position2 += roll2
                position2 = CHUTES_LADDERS.get(position2, position2)
                
                ### logging positions/roll data
                position1_list += [position1]
                position2_list += [position2]
                position += [[position1, position2]]
                roll_list += [[roll1, roll2]]

                
                
                ### ends game
                if position1 >= 100:
                    final_position += [position]
                    winner += [0.0]
                    turns_list += [turns]
                    rolls_list += [roll_list]
                    log_p_xz = np.array(np.log(log_p_xz))
                    log_list += list(log_p_xz)
                    
                    
                    position1_list += ["Player 1 Wins"]
                    position2_list += ["Player 1 Wins"]
                    continue
                    
                elif position2 >= 100:
                    final_position += [position]
                    rolls_list += [roll_list]
                    winner += [1.0] 
                    turns_list += [turns]     
                    log_p_xz = np.array(np.log(log_p_xz))   
                    log_list += list(log_p_xz)

                    position1_list += ["Player 2 Wins"]
                    position2_list += ["Player 2 Wins"]
                    continue
                    
        return(np.array(log_list)) #turns_list)

    def d_simulate(self, theta):
        d_simulate = ag.grad(self.simulate)
        return(d_simulate(theta))

    
    def get_theta_n(self, n_theta): #used to get theta values for rvs_ratio, if theta is not specified
        all_theta = []
        for i in range(n_theta):
            x = np.random.dirichlet(np.ones(6), size=1)
            all_theta += [x.tolist()[0]]
        return(all_theta)
    
    
    def rvs(self, theta=weights, n=n, random_state=None): 
        return(self.simulate(n=n, theta=theta))
        
    
    def rvs_score(self, theta=weights,n=n, random_state=None):
        all_x = []
        all_t_xz = []
        
        for i in range(n):
            #_, x = self.simulate(theta=theta, n=n)
            t_xz = self.d_simulate(theta=theta, n=n)

            #all_x.append(x)
            all_t_xz.append(t_xz)
            
        #all_x = np.array(all_x)
        all_t_xz = np.array(all_t_xz)
        return(all_t_xz)

    
    
    #theta_n is a list of arrays of probabilities
    def rvs_ratio(self, theta0=weights, theta_n=None, n=1, random_state=None): 
        
        if theta_n == None:
            theta_n = self.get_theta_n(1)
        elif type(theta_n) == int:
            theta_n = self.get_theta_n(theta_n)
                
        turns_list = [] #number of turns per game
        winner = [] #which player wins each game
        rolls_list = []

        log_p0_xz = []
        log_pn_xz = [[]  for i in range(len(theta_n))]
        all_log_r_xz = [[] for i in range(len(theta_n))]
        all_r_xz = [[] for i in range(len(theta_n))]
        

        counter = 0

        while counter < n:
            ### setting up game
            roll_theta0_array = np.array([1.0])
            
            array_dict = {}
            for i in range(len(theta_n)):
                array_dict[i+1] = np.array([1.0])
                
            roll_list = []
            position = []
            final_position = []
            position1 = 0
            position2 = 0
            turns = 0
            counter += 1


            while position1 < 100 and position2 < 100:
                turns += 1
                random_number1 = np.random.rand()
                random_number2 = np.random.rand()
                ### player_1's turn
                if random_number1 <= theta0[0]:
                    roll1 = 1
                    roll_theta0_array *= theta0[0]
                    for i in range(len(theta_n)):
                        array_dict[i+1] *=  theta_n[i][0]
                        
                elif random_number1 <= theta0[0] + theta0[1]:
                    roll1 = 2
                    roll_theta0_array *= theta0[1]
                    for i in range(len(theta_n)):
                        array_dict[i+1] *=  theta_n[i][1]
                        
                elif random_number1 <= theta0[0] + theta0[1] + theta0[2]:
                    roll1 = 3
                    roll_theta0_array *= theta0[2]
                    for i in range(len(theta_n)):
                        array_dict[i+1] *=  theta_n[i][2]
                        
                elif random_number1 <= theta0[0] + theta0[1] + theta0[2] + theta0[3]:
                    roll1 = 4
                    roll_theta0_array *= theta0[3]
                    for i in range(len(theta_n)):
                        array_dict[i+1] *=  theta_n[i][3]
                        
                elif random_number1 <= theta0[0] + theta0[1] + theta0[2] + theta0[3]+ theta0[4]:
                    roll1 = 5
                    roll_theta0_array *= theta0[4]
                    for i in range(len(theta_n)):
                        array_dict[i+1] *=  theta_n[i][4]
                        
                else:
                    roll1 = 6
                    roll_theta0_array *= theta0[5]
                    for i in range(len(theta_n)):
                        array_dict[i+1] *=  theta_n[i][5]
                    
                    
                position1 += roll1
                position1 = CHUTES_LADDERS.get(position1, position1)

                ### player_2's turn
                if random_number2 <= theta0[0]:
                    roll2 = 1
                    roll_theta0_array *= theta0[0]
                    for i in range(len(theta_n)):
                        array_dict[i+1] *=  theta_n[i][0]
                    
                    
                    
                elif random_number2 <= theta0[0] + theta0[1]:
                    roll2 = 2
                    roll_theta0_array *= theta0[1]
                    for i in range(len(theta_n)):
                        array_dict[i+1] *=  theta_n[i][1]
                    
                    
                    
                elif random_number2 <= theta0[0] + theta0[1] + theta0[2]:
                    roll2 = 3
                    roll_theta0_array *= theta0[2]
                    for i in range(len(theta_n)):
                        array_dict[i+1] *=  theta_n[i][2]
                    
                    
                    
                elif random_number2 <= theta0[0] + theta0[1] + theta0[2] + theta0[3]:
                    roll2 = 4
                    roll_theta0_array *= theta0[3]
                    for i in range(len(theta_n)):
                        array_dict[i+1] *=  theta_n[i][3]
                    
                    
                    
                elif random_number2 <= theta0[0] + theta0[1] + theta0[2] + theta0[3]+ theta0[4]:
                    roll2 = 5
                    roll_theta0_array *= theta0[4]
                    for i in range(len(theta_n)):
                        array_dict[i+1] *=  theta_n[i][4]
                    
                    
                    
                else:
                    roll2 = 6
                    roll_theta0_array *= theta0[5]
                    for i in range(len(theta_n)):
                        array_dict[i+1] *=  theta_n[i][5]           
                
                position2 += roll2
                position2 = CHUTES_LADDERS.get(position2, position2)

                ### logging positions/roll data
                position += [[position1, position2]]
                roll_list += [[roll1, roll2]]
                



                ### ends game
                if position1 >= 100:
                    final_position += [position]
                    winner += [0]
                    turns_list += [turns]
                    rolls_list += [roll_list]

                    log_p0_xz += list((np.log(roll_theta0_array)))
                    
                    for i in range(len(log_pn_xz)):
                        log_pn_xz[i] += list((np.log(array_dict[i+1])))
                        all_log_r_xz[i] += list(np.log(roll_theta0_array) - np.log(array_dict[i+1]))    
                        all_r_xz[i] += list(np.exp(np.log(roll_theta0_array) - np.log(array_dict[i+1])))

                    continue
                elif position2 >= 100:
                    final_position += [position]
                    winner += [1] 
                    rolls_list += [roll_list]
                    turns_list += [turns]     

                    log_p0_xz += list((np.log(roll_theta0_array)))
                    for i in range(len(log_pn_xz)):
                        log_pn_xz[i] += list((np.log(array_dict[i+1])))
                        all_log_r_xz[i] += list(np.log(roll_theta0_array) - np.log(array_dict[i+1]))    
                        all_r_xz[i] += list(np.exp(np.log(roll_theta0_array) - np.log(array_dict[i+1])))

                    continue


        return(log_pn_xz, all_r_xz)    
  
    
    

    def rvs_ratio_score(self, theta0=weights, theta1=None, n=n, theta=weights, random_state=None):
        turns, ratio, log_p0_xz, log_p1_xz = self.rvs_ratio(theta0=theta0, theta1=theta1, n=n)
        d_func = ag.grad_and_aux(self.simulate)
        dlog_p_xz= d_func(theta=weights)
        return(turns, ratio, score, log_p0_xz, log_p1_xz)
        


test = Chutes_Ladders()



In [136]:
test.d_simulate(theta=[1/6,1/6,1/6,1/6,1/6,1/6])

[array(0.0),
 array(18.0),
 array(5.999999999999999),
 array(24.0),
 array(29.999999999999996),
 array(18.0)]